In [1]:
%load_ext autoreload
%autoreload 2

import os as os
import numpy as np

import parmed as pmd
import pytraj as pt

In [2]:
import logging
import datetime as dt
d_date = dt.datetime.now()
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('Started logging...')

import paprika
print(paprika.__version__)

from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint
from paprika.restraints import amber_restraint_line
from paprika.restraints import create_window_list

from paprika.utils import make_window_dirs

from paprika.analysis import fe_calc

2018-06-04 11:06:55 AM Started logging...
2018-06-04 11:06:55 AM OpenMM support: Yes


2018-06-01_16:47:01_-0700-9944705-0.0.3


# Variables

In [3]:
prefix = '/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/'

# Set up original restraints (from `02-confirm-restraints`) and add information for `.phase['release']` instead of setting up dedicated release-only restraints and then fussing with `collect_data()` in `analysis.py`

In [4]:
hg = pmd.load_file('systems/a-bam-p/confirm-original/a000/full.hmr.topo',
                   'systems/a-bam-p/confirm-original/a000/full.crds',
                    structure=True)

dummy_anchors = [':1', ':2', ':3']
host_anchors  = [':4@O3', ':6@C1', ':8@C6']
guest_anchors = [':10@C4', ':10@N1']

In [5]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

pull_string = '0.00 0.40 0.80 1.20 1.60 2.00 2.40 2.80 3.20 3.60 4.00 4.40 4.80 5.20 5.60 6.00 6.40 6.80 7.20 7.60 8.00 8.40 8.80 9.20 9.60 10.00 10.40 10.80 11.20 11.60 12.00 12.40 12.80 13.20 13.60 14.00 14.40 14.80 15.20 15.60 16.00 16.40 16.80 17.20 17.60 18.00'
pull_distances = [float(i) + 6.00 for i in pull_string.split()]

release_fractions = attach_fractions[::-1]

windows = [len(attach_fractions), len(pull_distances), len(release_fractions)]
print(f'There are {windows} windows in this attach-pull-release calculation.')

There are [15, 46, 15] windows in this attach-pull-release calculation.


## Restraint definitions

### Static restraints connecting the host and dummy atoms

In [6]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0
static_restraint_angle_fc = 100.0

### Guest restraints connecting the guest and dummy atoms

In [7]:
guest_restraint_atoms = [[dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[1], dummy_anchors[0], guest_anchors[0]],
                         [dummy_anchors[0], guest_anchors[0], guest_anchors[1]],
                        ]

guest_restraint_targets = [6.0,
                           180.0,
                           180.0
                          ]
guest_restraint_target_final = [24.0, 180.0, 180.0]
guest_restraint_distance_fc = 5.0
guest_restraint_angle_fc = 100.0

### Host conformational restraints

In [8]:
host_conformational_template = [['O5', 'C1', 'O1', 'C4'],
                                ['C1', 'O1', 'C4', 'C5']
                               ]

host_residues = len(hg[':MGO'].residues)
first_host_residue = hg[':MGO'].residues[0].number + 1
conformational_restraint_atoms = []
conformational_restraint_targets = []
conformational_restraint_fc = 6.0

for n in range(first_host_residue, host_residues + first_host_residue):
    if n + 1 < host_residues + first_host_residue:
        next_residue = n + 1
    else:
        next_residue = first_host_residue
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[0][0]}',
         f':{n}@{host_conformational_template[0][1]}',
         f':{n}@{host_conformational_template[0][2]}',
         f':{next_residue}@{host_conformational_template[0][3]}'
        ])
    conformational_restraint_targets.append(104.30)
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[1][0]}',
         f':{n}@{host_conformational_template[1][1]}',
         f':{next_residue}@{host_conformational_template[1][2]}',
         f':{next_residue}@{host_conformational_template[1][3]}'
        ])
    conformational_restraint_targets.append(-108.8)

### Guest wall restraints

In [9]:
guest_wall_template = [
    ['O2', guest_anchors[0]],
    ['O6', guest_anchors[0]]
]

guest_wall_restraint_atoms = []
guest_wall_restraint_targets = []
guest_wall_restraint_angle_fc = 500.0
guest_wall_restraint_distance_fc = 50.0

for n in range(first_host_residue, host_residues + first_host_residue):
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[0][0]}',
         f'{guest_wall_template[0][1]}',
        ])
    guest_wall_restraint_targets.append(11.3)
    guest_wall_restraint_atoms.append(
        [f':{n}@{guest_wall_template[1][0]}',
         f'{guest_wall_template[1][1]}',
        ])
    guest_wall_restraint_targets.append(13.3)

guest_wall_restraint_atoms.append([dummy_anchors[1],
                                 guest_anchors[0],
                                 guest_anchors[1]])
guest_wall_restraint_targets.append(80.0)

## Initialize the restraints

### Static restraints

In [10]:
static_restraints = []
for index, atoms in enumerate(static_restraint_atoms):    
    this = static_DAT_restraint(restraint_mask_list=atoms,
                        num_window_list=windows, 
                        ref_structure=hg,
                        force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                        amber_index=True)
    static_restraints.append(this)

2018-06-04 11:06:57 AM Calculating attach targets and force constants...
2018-06-04 11:06:57 AM Attach, Method #1
2018-06-04 11:06:57 AM Calculating pull targets and force constants...
2018-06-04 11:06:57 AM Pull, Method #1
2018-06-04 11:06:57 AM Calculating release targets and force constants...
2018-06-04 11:06:57 AM Release, Method #1
2018-06-04 11:06:57 AM Number of attach windows = 15
2018-06-04 11:06:57 AM Number of pull windows = 46
2018-06-04 11:06:57 AM Number of release windows = 15
2018-06-04 11:06:57 AM Assigning atom indices...
2018-06-04 11:06:57 AM There are 1 atoms in the mask :1  ...
2018-06-04 11:06:57 AM There are 1 atoms in the mask :4@O3  ...
2018-06-04 11:06:57 AM Calculating attach targets and force constants...
2018-06-04 11:06:57 AM Attach, Method #1
2018-06-04 11:06:57 AM Calculating pull targets and force constants...
2018-06-04 11:06:57 AM Pull, Method #1
2018-06-04 11:06:57 AM Calculating release targets and force constants...
2018-06-04 11:06:57 AM Release

### Guest restraints

In [11]:
guest_restraints = []
for index, atoms in enumerate(guest_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_final'] = guest_restraint_angle_fc
        this.release['fc_final'] = guest_restraint_angle_fc
    else:
        this.attach['fc_final'] = guest_restraint_distance_fc
        this.release['fc_final'] = guest_restraint_distance_fc
    this.attach['target'] = guest_restraint_targets[index]
    this.attach['fraction_list'] = attach_fractions
    
    this.pull['target_final'] = guest_restraint_target_final[index]
    this.pull['num_windows'] = windows[1]
    
    this.release['target'] = guest_restraint_targets[index]
    # Keep the guest restraints on during release.
    this.release['fraction_list'] = [1.0] * windows[2]
    
    this.initialize()
    guest_restraints.append(this)

2018-06-04 11:06:59 AM Calculating attach targets and force constants...
2018-06-04 11:06:59 AM Attach, Method #3
2018-06-04 11:06:59 AM Calculating pull targets and force constants...
2018-06-04 11:06:59 AM Pull, Method #1
2018-06-04 11:06:59 AM Calculating release targets and force constants...
2018-06-04 11:06:59 AM Release, Method #3
2018-06-04 11:06:59 AM Number of attach windows = 15
2018-06-04 11:06:59 AM Number of pull windows = 46
2018-06-04 11:06:59 AM Number of release windows = 15
2018-06-04 11:06:59 AM Assigning atom indices...
2018-06-04 11:06:59 AM There are 1 atoms in the mask :1  ...
2018-06-04 11:06:59 AM There are 1 atoms in the mask :10@C4  ...
2018-06-04 11:06:59 AM Calculating attach targets and force constants...
2018-06-04 11:06:59 AM Attach, Method #3
2018-06-04 11:06:59 AM Calculating pull targets and force constants...
2018-06-04 11:06:59 AM Pull, Method #1
2018-06-04 11:06:59 AM Calculating release targets and force constants...
2018-06-04 11:06:59 AM Releas

### Host conformational restraints

In [12]:
conformational_restraints = []
for index, atoms in enumerate(conformational_restraint_atoms): 
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    this.mask3 = atoms[2]
    this.mask4 = atoms[3]
    
    this.attach['fraction_list'] = attach_fractions
    this.attach['target'] = conformational_restraint_targets[index]
    this.attach['fc_final'] = conformational_restraint_fc
    this.pull['target_final'] = conformational_restraint_targets[index]
    this.pull['num_windows'] = windows[1]
    
    this.release['fraction_list'] = release_fractions
    this.release['target'] = conformational_restraint_targets[index]
    this.release['fc_final'] = conformational_restraint_fc
    
    this.initialize()
    conformational_restraints.append(this)

2018-06-04 11:06:59 AM Calculating attach targets and force constants...
2018-06-04 11:06:59 AM Attach, Method #3
2018-06-04 11:06:59 AM Calculating pull targets and force constants...
2018-06-04 11:06:59 AM Pull, Method #1
2018-06-04 11:06:59 AM Calculating release targets and force constants...
2018-06-04 11:06:59 AM Release, Method #3
2018-06-04 11:06:59 AM Number of attach windows = 15
2018-06-04 11:06:59 AM Number of pull windows = 46
2018-06-04 11:06:59 AM Number of release windows = 15
2018-06-04 11:06:59 AM Assigning atom indices...
2018-06-04 11:06:59 AM There are 1 atoms in the mask :4@O5  ...
2018-06-04 11:06:59 AM There are 1 atoms in the mask :4@C1  ...
2018-06-04 11:06:59 AM There are 1 atoms in the mask :4@O1  ...
2018-06-04 11:06:59 AM There are 1 atoms in the mask :5@C4  ...
2018-06-04 11:06:59 AM Calculating attach targets and force constants...
2018-06-04 11:06:59 AM Attach, Method #3
2018-06-04 11:06:59 AM Calculating pull targets and force constants...
2018-06-04 1

2018-06-04 11:07:01 AM Pull, Method #1
2018-06-04 11:07:01 AM Calculating release targets and force constants...
2018-06-04 11:07:01 AM Release, Method #3
2018-06-04 11:07:01 AM Number of attach windows = 15
2018-06-04 11:07:01 AM Number of pull windows = 46
2018-06-04 11:07:01 AM Number of release windows = 15
2018-06-04 11:07:01 AM Assigning atom indices...
2018-06-04 11:07:01 AM There are 1 atoms in the mask :9@O5  ...
2018-06-04 11:07:01 AM There are 1 atoms in the mask :9@C1  ...
2018-06-04 11:07:01 AM There are 1 atoms in the mask :9@O1  ...
2018-06-04 11:07:01 AM There are 1 atoms in the mask :4@C4  ...
2018-06-04 11:07:01 AM Calculating attach targets and force constants...
2018-06-04 11:07:01 AM Attach, Method #3
2018-06-04 11:07:01 AM Calculating pull targets and force constants...
2018-06-04 11:07:01 AM Pull, Method #1
2018-06-04 11:07:01 AM Calculating release targets and force constants...
2018-06-04 11:07:01 AM Release, Method #3
2018-06-04 11:07:01 AM Number of attach wi

### Guest wall restraints

In [13]:
wall_restraints = []
for index, atoms in enumerate(guest_wall_restraint_atoms): 
    if len(atoms) > 2:
        angle = True
    else:
        angle = False
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    if angle:
        this.mask3 = atoms[2]
        this.attach['fc_initial'] = guest_wall_restraint_angle_fc
        this.attach['fc_final'] = guest_wall_restraint_angle_fc
        this.custom_restraint_values['rk2'] = 500.0
        this.custom_restraint_values['rk3'] = 0.0
    else:
        this.attach['fc_initial'] = guest_wall_restraint_distance_fc
        this.attach['fc_final'] = guest_wall_restraint_distance_fc
        this.custom_restraint_values['rk2'] = 50.0
        this.custom_restraint_values['rk3'] = 50.0
        this.custom_restraint_values['r1'] = 0.0
        this.custom_restraint_values['r2'] = 0.0

    this.attach['target'] = guest_wall_restraint_targets[index]
    this.attach['num_windows'] = len(attach_fractions)
       
    this.initialize()
    wall_restraints.append(this)

2018-06-04 11:07:01 AM Calculating attach targets and force constants...
2018-06-04 11:07:01 AM Attach, Method #1
2018-06-04 11:07:01 AM Calculating pull targets and force constants...
2018-06-04 11:07:01 AM No restraint info set for the pull phase! Skipping...
2018-06-04 11:07:01 AM Calculating release targets and force constants...
2018-06-04 11:07:01 AM No restraint info set for the release phase! Skipping...
2018-06-04 11:07:01 AM Number of attach windows = 15
2018-06-04 11:07:01 AM This restraint will be skipped in the pull phase
2018-06-04 11:07:01 AM This restraint will be skipped in the release phase
2018-06-04 11:07:01 AM Assigning atom indices...
2018-06-04 11:07:01 AM There are 1 atoms in the mask :4@O2  ...
2018-06-04 11:07:01 AM There are 1 atoms in the mask :10@C4  ...
2018-06-04 11:07:01 AM Calculating attach targets and force constants...
2018-06-04 11:07:01 AM Attach, Method #1
2018-06-04 11:07:01 AM Calculating pull targets and force constants...
2018-06-04 11:07:01 A

2018-06-04 11:07:02 AM Calculating release targets and force constants...
2018-06-04 11:07:02 AM No restraint info set for the release phase! Skipping...
2018-06-04 11:07:02 AM Number of attach windows = 15
2018-06-04 11:07:02 AM This restraint will be skipped in the pull phase
2018-06-04 11:07:02 AM This restraint will be skipped in the release phase
2018-06-04 11:07:02 AM Assigning atom indices...
2018-06-04 11:07:02 AM There are 1 atoms in the mask :9@O2  ...
2018-06-04 11:07:02 AM There are 1 atoms in the mask :10@C4  ...
2018-06-04 11:07:02 AM Calculating attach targets and force constants...
2018-06-04 11:07:02 AM Attach, Method #1
2018-06-04 11:07:02 AM Calculating pull targets and force constants...
2018-06-04 11:07:02 AM No restraint info set for the pull phase! Skipping...
2018-06-04 11:07:02 AM Calculating release targets and force constants...
2018-06-04 11:07:02 AM No restraint info set for the release phase! Skipping...
2018-06-04 11:07:02 AM Number of attach windows = 15

In [14]:
window_list = create_window_list(guest_restraints)

2018-06-04 11:07:02 AM All restraints are "continuous_apr" style.
2018-06-04 11:07:02 AM Restraints appear to be consistent


In [15]:
# files = ['full.hmr.topo',
#         'full.crds',
#         ]

# for window_index, window in enumerate([i for i in window_list if i[0] == 'r']):
#     for file in files:
#         if os.path.isfile(os.path.join(prefix, 'confirm-original', window, file)):
#             os.remove(os.path.join(prefix, 'confirm-original', window, file))
#         try:
#             os.symlink(os.path.join('../', 'p045', file),
#                        os.path.join(prefix, 'confirm-original', window, file))        
#         except:
#             pass

In [16]:
# %pdb
# print('Writing restratint file in each window...')
# for window in window_list:
#     with open(os.path.join(prefix, 'confirm-original', window, 'disang.rest'), 'w') as file:
#         for restraint in static_restraints + guest_restraints + \
#                           conformational_restraints + wall_restraints:
#             if window[0] == 'a':
#                 phase = 'attach'
#             if window[0] == 'p':
#                 phase = 'pull'
#             if window[0] == 'r':
#                 phase = 'release'

#             try:
#                 string = amber_restraint_line(restraint, phase, int(window[1:]))
#             except:
#                 # Hack to get around trying to access
#                 # --> 630         'r2': restraint.phase[phase]['targets'][window],
#                 # for wall restraints, during pull.
#                 continue
#             if string is not None:
#                 file.write(string)

In [17]:
structure = pt.load(
    os.path.join('systems', 'a-bam-p', 'confirm-original', 'a000', 'full.crds'),
    os.path.join('systems', 'a-bam-p', 'confirm-original', 'a000', 'full.hmr.topo'))

stripped = structure.strip(':WAT,:Na+,:Cl-')

In [20]:
analyze = fe_calc()
analyze.prmtop = stripped.topology
analyze.trajectory = 'prod.*.nc'
analyze.path = os.path.join('/home', 'dslochower', 'kirkwood', 'projects', 'smirnoff-host-guest-simulations', 
                            'systems', 'a-bam-p', 'confirm-original')


In [21]:
analyze.restraint_list = guest_restraints + conformational_restraints
analyze.collect_data()
analyze.methods = ['ti-block']
analyze.quicker_ti_matrix = True
analyze.bootcycles = 1000
analyze.compute_free_energy()

2018-06-04 11:07:53 AM Replacing /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a014 with /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p000 in ['/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a000', '/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a001', '/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a002', '/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a003', '/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a004', '/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a005', '/home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original

2018-06-04 11:09:14 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a001...
2018-06-04 11:09:14 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a001...
2018-06-04 11:09:14 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a002...
2018-06-04 11:09:14 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a002...
2018-06-04 11:09:14 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a002...
2018-06-04 11:09:14 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a002...
2018-06-04 11:09:14 AM Reading restraint data for /h

2018-06-04 11:09:16 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a005...
2018-06-04 11:09:16 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a005...
2018-06-04 11:09:16 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a005...
2018-06-04 11:09:16 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a005...
2018-06-04 11:09:16 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a005...
2018-06-04 11:09:16 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a005...
2018-06-04 11:09:16 AM Reading restraint data for /h

2018-06-04 11:09:18 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a008...
2018-06-04 11:09:18 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a008...
2018-06-04 11:09:18 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a008...
2018-06-04 11:09:18 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a009...
2018-06-04 11:09:18 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a009...
2018-06-04 11:09:18 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a009...
2018-06-04 11:09:18 AM Reading restraint data for /h

2018-06-04 11:09:20 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a012...
2018-06-04 11:09:20 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a012...
2018-06-04 11:09:20 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a012...
2018-06-04 11:09:20 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a012...
2018-06-04 11:09:20 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a012...
2018-06-04 11:09:20 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/a012...
2018-06-04 11:09:20 AM Reading restraint data for /h

2018-06-04 11:09:23 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p011...
2018-06-04 11:09:23 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p012...
2018-06-04 11:09:23 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p013...
2018-06-04 11:09:23 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p014...
2018-06-04 11:09:23 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p015...
2018-06-04 11:09:23 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p016...
2018-06-04 11:09:23 AM Reading restraint data for /h

2018-06-04 11:09:33 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r013...
2018-06-04 11:09:33 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r013...
2018-06-04 11:09:33 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r013...
2018-06-04 11:09:34 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r013...
2018-06-04 11:09:34 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r013...
2018-06-04 11:09:34 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r013...
2018-06-04 11:09:34 AM Reading restraint data for /h

2018-06-04 11:09:47 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r009...
2018-06-04 11:09:48 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r009...
2018-06-04 11:09:48 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r009...
2018-06-04 11:09:48 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r008...
2018-06-04 11:09:49 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r008...
2018-06-04 11:09:49 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r008...
2018-06-04 11:09:49 AM Reading restraint data for /h

2018-06-04 11:10:01 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r004...
2018-06-04 11:10:02 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r004...
2018-06-04 11:10:02 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r004...
2018-06-04 11:10:02 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r004...
2018-06-04 11:10:03 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r004...
2018-06-04 11:10:03 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/r004...
2018-06-04 11:10:03 AM Reading restraint data for /h

2018-06-04 11:10:38 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p045...
2018-06-04 11:10:43 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p045...
2018-06-04 11:10:48 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p045...
2018-06-04 11:10:53 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p045...
2018-06-04 11:10:59 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p045...
2018-06-04 11:11:04 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/p045...
2018-06-04 11:11:10 AM Reading restraint data for /h

In [28]:
analyze.compute_ref_state_work([guest_restraints[0], guest_restraints[1], None, None, guest_restraints[2], None])

In [23]:
analyze.results['attach']['ti-block']['fe']
analyze.results['pull']['ti-block']['fe']
analyze.results['release']['ti-block']['fe']

-8.0361427085368646

In [24]:
from paprika.analysis import save

In [29]:
save(analyze, 'confirm-results.json')

In [30]:
analyze.results

{'attach': {'ti-block': {'fe': 10.426596177106555,
   'sem': 0.26876276708660191,
   'fe_matrix': array([[  0.        ,   0.50863948,   0.8519315 , ...,   7.62406471,
             9.2182979 ,  10.42659618],
          [ -0.50863948,   0.        ,   0.        , ...,   0.        ,
             0.        ,   9.91241076],
          [ -0.8519315 ,  -0.        ,   0.        , ...,   0.        ,
             0.        ,   9.573151  ],
          ..., 
          [ -7.62406471,  -0.        ,  -0.        , ...,   0.        ,
             0.        ,   2.79323708],
          [ -9.2182979 ,  -0.        ,  -0.        , ...,  -0.        ,
             0.        ,   1.19451169],
          [-10.42659618,  -9.91241076,  -9.573151  , ...,  -2.79323708,
            -1.19451169,   0.        ]]),
   'sem_matrix': array([[ 0.        ,  0.14691728,  0.24590286, ...,  0.26495521,
            0.26675265,  0.26876277],
          [ 0.14691728,  0.        ,  0.        , ...,  0.        ,
            0.        ,  0.

In [31]:
print(f"{'Attach': <10} {analyze.results['attach']['ti-block']['fe']:+04.2f} +/- {analyze.results['attach']['ti-block']['sem']:2.2f}")
print(f"{'Pull': <10} {analyze.results['pull']['ti-block']['fe']:+04.2f} +/- {analyze.results['pull']['ti-block']['sem']:2.2f}")
print(f"{'Release': <10} {analyze.results['release']['ti-block']['fe']:+04.2f} +/- {analyze.results['release']['ti-block']['sem']:2.2f}")
print(f"{'Reference': <10} {analyze.results['ref_state_work']:+04.2f}")

fe = analyze.results['attach']['ti-block']['fe'] + analyze.results['pull']['ti-block']['fe'] + analyze.results['release']['ti-block']['fe'] + analyze.results['ref_state_work']
sem = np.sqrt( (analyze.results['attach']['ti-block']['sem']**2 + \
                analyze.results['pull']['ti-block']['sem']**2 + \
                analyze.results['release']['ti-block']['sem']**2
               )
             )
print(f"{'Total': <10} {fe:+2.2f} +/- {sem:2.2f}")

Attach     +10.43 +/- 0.27
Pull       +5.44 +/- 0.35
Release    -8.04 +/- 0.12
Reference  -7.14
Total      +0.68 +/- 0.45
